# 2025 Ballot Proposal Geospatial Analysis

This notebook
* Part 1: Cleans the ballot results to prepare for spatial joining
* Part 2: Uses Geopandas for the spatial joining map

### Part 1: Ballot Name Cleaning

In [1]:
import geopandas as gpd
import pandas as pd
from geopandas import sjoin

In [2]:
df = pd.read_csv('../data/full_results_2025_copy.csv')

### Cleaning Values

Filter the `office` column to only display results for the proposals

In [3]:
ballot = df[df['office'].str.contains('Proposal Number', case=False, na=False)].copy()


In [4]:
ballot['office'].value_counts()

office
Proposal Number 1, an Amendment: Amendment to Allow Olympic Sports Complex In Essex County on State Forest Preserve Land                                        8676
Proposal Number 2, a Question: Proposed Charter Amendment: Fast Track Affordable Housing to Build More Affordable Housing Across the City                       8676
Proposal Number 3, a Question: Proposed Charter Amendment: Simplify Review of Modest Housing and Infrastructure Projects                                        8676
Proposal Number 4, a Question: Proposed Charter Amendment: Establish an Affordable Housing Appeals Board with Council, Borough, and Citywide Representation     8676
Proposal Number 5, a Question: Proposed Charter Amendment: Create a Digital City Map to Modernize City Operations                                               8676
Proposal Number 6, a Question: Proposed Charter Amendment: Move Local Elections to Presidential Election Years to Increase Voter Participation                  8676
Nam

Create a new column for proposals and renames the value using regex for readability


In [5]:
ballot["proposal_number"] = ballot["office"].str.extract(r"(Proposal Number \d+)") 
print(ballot["proposal_number"])

209568    Proposal Number 1
209569    Proposal Number 1
209570    Proposal Number 1
209571    Proposal Number 1
209572    Proposal Number 1
                ...        
261619    Proposal Number 6
261620    Proposal Number 6
261621    Proposal Number 6
261622    Proposal Number 6
261623    Proposal Number 6
Name: proposal_number, Length: 52056, dtype: object


Create a new column for ed match, combining the ad and ed values


In [6]:
ballot["district_combo"] = ballot.apply(
    lambda row: str(row["ad"]) + 
                (("00" + str(int(row["ed"]))) if int(row["ed"]) < 10
                 else ("0" + str(int(row["ed"]))) if int(row["ed"]) < 100
                 else str(int(row["ed"]))),
    axis=1
)
ballot.head()

,office,id,ad,ed,candidate,party,votes,proposal_number,district_combo
209568,"Proposal Number 1, an Amendment: Amendment to ...",27305,23,1,YES,NaN,367,Proposal Number 1,23001
209569,"Proposal Number 1, an Amendment: Amendment to ...",27305,23,1,NO,NaN,674,Proposal Number 1,23001
209570,"Proposal Number 1, an Amendment: Amendment to ...",27305,23,2,YES,NaN,304,Proposal Number 1,23002
209571,"Proposal Number 1, an Amendment: Amendment to ...",27305,23,2,NO,NaN,715,Proposal Number 1,23002
209572,"Proposal Number 1, an Amendment: Amendment to ...",27305,23,3,YES,NaN,106,Proposal Number 1,23003


Drop the ad and ed column and create a pivot table


In [7]:
ballot_results = (
    ballot.groupby(["proposal_number", "district_combo", "candidate"], as_index=False)["votes"]
    .sum()
    .pivot(index=["proposal_number", "district_combo"], columns="candidate", values="votes")
    .reset_index()
)
print(ballot_results)

candidate    proposal_number district_combo   NO  YES
0          Proposal Number 1          23001  674  367
1          Proposal Number 1          23002  715  304
2          Proposal Number 1          23003  249  106
3          Proposal Number 1          23004  712  321
4          Proposal Number 1          23005  732  311
...                      ...            ...  ...  ...
26023      Proposal Number 6          87051  153  116
26024      Proposal Number 6          87052   90   90
26025      Proposal Number 6          87053    0    0
26026      Proposal Number 6          87054    0    0
26027      Proposal Number 6          87055    0    0

[26028 rows x 4 columns]


### Calculate Vote Share Percentages

The next cell simplifies the proposal names:
*  Creates a yes and no percentage columns
*  Calculates percentages for each row


In [8]:

ballot_results["yes_pct"] = (
    ballot_results["YES"] / (ballot_results["YES"] + ballot_results["NO"]) * 100
).round(1)

ballot_results["no_pct"] = (
    ballot_results["NO"] / (ballot_results["YES"] + ballot_results["NO"]) * 100
).round(1)

print(ballot_results[["yes_pct", "no_pct"]])


candidate  yes_pct  no_pct
0             35.3    64.7
1             29.8    70.2
2             29.9    70.1
3             31.1    68.9
4             29.8    70.2
...            ...     ...
26023         43.1    56.9
26024         50.0    50.0
26025          NaN     NaN
26026          NaN     NaN
26027          NaN     NaN

[26028 rows x 2 columns]


In [9]:
ballot_results.head()

candidate,proposal_number,district_combo,NO,YES,yes_pct,no_pct
0,Proposal Number 1,23001,674,367,35.3,64.7
1,Proposal Number 1,23002,715,304,29.8,70.2
2,Proposal Number 1,23003,249,106,29.9,70.1
3,Proposal Number 1,23004,712,321,31.1,68.9
4,Proposal Number 1,23005,732,311,29.8,70.2


The next cell:
* Creates a function that returns a percentage category based on the yes_pct value
* Applies these categories into a new column for Datawrapper

In [10]:
def categorize_yes_pct(value):
    if value < 20:
        return "<20%"
    elif value <= 40:
        return "21-40%"
    elif value <= 60:
        return "41-60%"
    elif value <= 80:
        return "61-80%"
    else:
        return ">80%"

ballot_results["dw_category"] = ballot_results["yes_pct"].apply(categorize_yes_pct)


In [11]:
ballot_results.head()

candidate,proposal_number,district_combo,NO,YES,yes_pct,no_pct,dw_category
0,Proposal Number 1,23001,674,367,35.3,64.7,21-40%
1,Proposal Number 1,23002,715,304,29.8,70.2,21-40%
2,Proposal Number 1,23003,249,106,29.9,70.1,21-40%
3,Proposal Number 1,23004,712,321,31.1,68.9,21-40%
4,Proposal Number 1,23005,732,311,29.8,70.2,21-40%


## Part 2: Spatial Joining

In [12]:
gdf_ed = gpd.read_file("../data/nyed_25c_git/") # 2025 election district shapefile
gdf_nta = gpd.read_file("../data/nyed_25c_neighborhood_name_reproj4326_git.geojson") # shapefile with cleaned nta names

In [13]:
gdf_ed.head()

,ElectDist,Shape_Leng,Shape_Area,geometry
0,23001,24593.971411,2.779129e+07,"POLYGON ((1006386.279 144120.655, 1006261.842 ..."
1,23002,15531.629681,9.753402e+06,"POLYGON ((1009207.47 145106.094, 1009147.042 1..."
2,23003,41787.651728,3.452959e+07,"POLYGON ((1022349.755 145817.949, 1022404.165 ..."
3,23004,13616.939281,8.166450e+06,"POLYGON ((1025161.154 147197.743, 1025055.81 1..."
4,23005,10698.547734,5.077745e+06,"POLYGON ((1026261.273 147727.118, 1026236.365 ..."


In [14]:
gdf_nta.head()

,electdist,shape_leng,shape_area,ntaname,nta2020,borough_ed,ntatype,name,borough_neighborhood,geometry
0,44010,2802.649918,4.682287e+05,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676..."
1,24026,6057.253242,1.884009e+06,Richmond Hill,QN0902,Queens,0,Richmond Hill,Queens,"POLYGON ((-73.8266 40.69818, -73.82753 40.6979..."
2,43013,3492.011032,7.512185e+05,Crown Heights (North),BK0802,Brooklyn,0,Crown Heights,Brooklyn,"POLYGON ((-73.94195 40.67208, -73.94203 40.671..."
3,74004,4887.084317,9.105137e+05,Murray Hill-Kips Bay,MN0603,Manhattan,0,Kips Bay,Manhattan,"POLYGON ((-73.97032 40.74816, -73.97027 40.748..."
4,61064,2161.221386,1.632385e+05,Financial District-Battery Park City,MN0101,Manhattan,0,Battery Park City,Manhattan,"POLYGON ((-74.01662 40.71216, -74.01658 40.712..."


Checking to see what crs each shapefile uses

In [24]:
print(gdf_ed.crs)
print(gdf_nta.crs)

EPSG:2263
EPSG:4326


Reproject gdf_ed to match gdf_nta (we are using the latter as a custom map shapefile for Datawrapper)

In [26]:
gdf_ed = gdf_ed.to_crs(gdf_nta.crs)

In [27]:
print(gdf_ed.crs)
print(gdf_nta.crs)

EPSG:4326
EPSG:4326


Now you can run the spatial join

In [29]:
joined_nta_ed = gpd.sjoin(gdf_nta, gdf_ed, how="left", predicate="intersects")


In [30]:
joined_nta_ed.head()

,electdist_left,shape_leng_left,shape_area_left,ntaname_left,nta2020_left,borough_ed_left,ntatype_left,name_left,borough_neighborhood_left,geometry,index_right,electdist_right,shape_leng_right,shape_area_right,ntaname_right,nta2020_right,borough_ed_right,ntatype_right,name_right,borough_neighborhood_right
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",1013,52076,4015.630144,5.542606e+05,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",1144,52075,4962.454402,6.733991e+05,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",772,44008,5388.580784,1.055822e+06,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",380,44060,2247.278225,2.938833e+05,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",248,44005,2151.755030,2.638098e+05,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn


In [32]:
joined_nta_ed.columns

Index(['electdist_left', 'shape_leng_left', 'shape_area_left', 'ntaname_left',
       'nta2020_left', 'borough_ed_left', 'ntatype_left', 'name_left',
       'borough_neighborhood_left', 'geometry', 'index_right',
       'electdist_right', 'shape_leng_right', 'shape_area_right',
       'ntaname_right', 'nta2020_right', 'borough_ed_right', 'ntatype_right',
       'name_right', 'borough_neighborhood_right'],
      dtype='object')

convert the columns for merging to strings

In [37]:
joined_nta_ed["electdist_right"] = joined_nta_ed["electdist_right"].astype(str)
ballot_results["district_combo"] = ballot_results["district_combo"].astype(str)

In [38]:
final_gdf = joined_nta_ed.merge(
    ballot_results,
    left_on="electdist_right",
    right_on="district_combo",
    how="left"
)

In [57]:
final_gdf.head()

,electdist_left,shape_leng_left,shape_area_left,ntaname_left,nta2020_left,borough_ed_left,ntatype_left,nta,borough_neighborhood_left,geometry,...,ntatype_right,name_right,borough_neighborhood_right,proposal_number,district_combo,NO,YES,yes_pct,no_pct,dw_category
0,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",...,0,Prospect Heights,Brooklyn,Proposal Number 1,52076,241,305,55.9,44.1,41-60%
1,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",...,0,Prospect Heights,Brooklyn,Proposal Number 2,52076,175,378,68.4,31.6,61-80%
2,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",...,0,Prospect Heights,Brooklyn,Proposal Number 3,52076,175,370,67.9,32.1,61-80%
3,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",...,0,Prospect Heights,Brooklyn,Proposal Number 4,52076,203,336,62.3,37.7,61-80%
4,44010,2802.649918,468228.687496,Prospect Heights,BK0801,Brooklyn,0,Prospect Heights,Brooklyn,"POLYGON ((-73.96371 40.67753, -73.96353 40.676...",...,0,Prospect Heights,Brooklyn,Proposal Number 5,52076,45,503,91.8,8.2,>80%


In [58]:
final_gdf.columns

Index(['electdist_left', 'shape_leng_left', 'shape_area_left', 'ntaname_left',
       'nta2020_left', 'borough_ed_left', 'ntatype_left', 'nta',
       'borough_neighborhood_left', 'geometry', 'index_right',
       'electdist_right', 'shape_leng_right', 'shape_area_right',
       'ntaname_right', 'nta2020_right', 'borough_ed_right', 'ntatype_right',
       'name_right', 'borough_neighborhood_right', 'proposal_number',
       'district_combo', 'NO', 'YES', 'yes_pct', 'no_pct', 'dw_category'],
      dtype='object')

Rename the `name_left` column

In [65]:
final_gdf = final_gdf.rename(columns={"name_left": "nta"})


Clean up the dataframe and drop the extraneous columns

In [68]:
cols_to_keep = [
    "district_combo",       # for tooltip
    "nta",         # for tooltip title
    "proposal_number", # for filtering
    "YES", "NO",       # vote counts
    "yes_pct", "no_pct", "dw_category"  # mapping + tooltip
]

final_gdf_reduced = final_gdf[cols_to_keep].copy()

Set up a loop that iterates through each proposal and created multiple csvs to export

In [69]:
for i in range(1, 7):
    proposal_df = final_gdf_reduced[
        final_gdf_reduced["proposal_number"] == f"Proposal Number {i}"
    ]
    proposal_df.to_csv(f"../output/proposal_{i}_results_2025.csv", index=False)
    print(f"Exported proposal_{i}_results_2025.csv with {len(proposal_df)} rows")

Exported proposal_1_results_2025.csv with 29637 rows
Exported proposal_2_results_2025.csv with 29637 rows
Exported proposal_3_results_2025.csv with 29637 rows
Exported proposal_4_results_2025.csv with 29637 rows
Exported proposal_5_results_2025.csv with 29637 rows
Exported proposal_6_results_2025.csv with 29637 rows
